In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [22]:
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

from keras.layers import Dense, Conv2D, MaxPooling2D, InputLayer, Flatten
from keras.models import Model, Sequential

import random

In [2]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

In [9]:
!unzip -q ../input/dogs-vs-cats/train.zip

In [12]:
os.mkdir('train/cat')
os.mkdir('train/dog')

In [14]:
max_imgs = 1000
cats = 0
dogs = 0

for file in os.walk('train'):
    filenames = file[2]
    for filename in filenames:
        if filename.find('cat') != -1:
            if cats < max_imgs:
                os.replace('train/' + filename, 'train/cat/' + filename[4:])
                cats += 1
        elif filename.find('dog') != -1:
            if dogs < max_imgs:
                os.replace('train/' + filename, 'train/dog/' + filename[4:])
                dogs += 1

In [15]:
categories = [x[0] for x in os.walk('train') if x[0]][1:]

['train/dog', 'train/cat']

In [16]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [17]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

In [18]:
random.shuffle(data)
train_split = 0.8
idx_test = int((train_split) * len(data))

data_train = data[:idx_test]
data_test = data[idx_test:]

In [19]:
x_train, y_train = np.array([t["x"] for t in data_train]), [t["y"] for t in data_train]
x_test, y_test = np.array([t["x"] for t in data_test]), [t["y"] for t in data_test]

In [20]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [28]:
def create_model(
    filters=[20, 20],
    pool_size=[(10, 10), (10, 10)],
    dense_size=[400, 400]):
    
    model = Sequential()

    model.add(InputLayer(input_shape=x_train.shape[1:]))
    model.add(Conv2D(filters[0], kernel_size=3))
    model.add(Conv2D(filters[1], kernel_size=3))
    model.add(MaxPooling2D(pool_size=pool_size[0]))
    model.add(Conv2D(filters[0], kernel_size=3))
    model.add(Conv2D(filters[1], kernel_size=3))
    model.add(MaxPooling2D(pool_size=pool_size[0]))
    model.add(Flatten())
    model.add(Dense(400, activation='relu'))
    model.add(Dense(400, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [29]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 222, 222, 20)      560       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 220, 220, 20)      3620      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 20)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 20)        3620      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 18, 18, 20)        3620      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 20)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20)               

In [25]:
model.fit(x_train, y_train, batch_size=10, epochs=10)

Epoch 1/10
160/160 [==============================] - 74s 458ms/step - loss: 0.7366 - accuracy: 0.5604
Epoch 2/10
160/160 [==============================] - 71s 444ms/step - loss: 0.5997 - accuracy: 0.6741
Epoch 3/10
160/160 [==============================] - 71s 443ms/step - loss: 0.5111 - accuracy: 0.7574
Epoch 4/10
160/160 [==============================] - 70s 439ms/step - loss: 0.4043 - accuracy: 0.8219
Epoch 5/10
160/160 [==============================] - 72s 452ms/step - loss: 0.2713 - accuracy: 0.8955
Epoch 6/10
160/160 [==============================] - 83s 521ms/step - loss: 0.2189 - accuracy: 0.9121
Epoch 7/10
160/160 [==============================] - 74s 464ms/step - loss: 0.0838 - accuracy: 0.9763
Epoch 8/10
160/160 [==============================] - 74s 460ms/step - loss: 0.0299 - accuracy: 0.9912
Epoch 9/10
160/160 [==============================] - 73s 456ms/step - loss: 0.1099 - accuracy: 0.9554
Epoch 10/10
160/160 [==============================] - 71s 445ms/step - l

In [26]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

In [27]:
# Метрики качества на тестовой выборке
print('LOSS', loss)
print('ACCURACY', accuracy)

LOSS 1.4530487060546875
ACCURACY 0.7074999809265137


### Точночть 71%

# VGG16

In [30]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

553467904/553467096 [==============================] - 7s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [31]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)

model_new.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [32]:
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

In [33]:
model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [34]:
model_new.fit(x_train, y_train, batch_size=10, epochs=10)

Epoch 1/10
160/160 [==============================] - 304s 2s/step - loss: 0.5062 - accuracy: 0.7464
Epoch 2/10
160/160 [==============================] - 305s 2s/step - loss: 0.3404 - accuracy: 0.8382
Epoch 3/10
160/160 [==============================] - 307s 2s/step - loss: 0.3207 - accuracy: 0.8529
Epoch 4/10
160/160 [==============================] - 303s 2s/step - loss: 0.2798 - accuracy: 0.8804
Epoch 5/10
160/160 [==============================] - 301s 2s/step - loss: 0.2274 - accuracy: 0.9016
Epoch 6/10
160/160 [==============================] - 303s 2s/step - loss: 0.2841 - accuracy: 0.8686
Epoch 7/10
160/160 [==============================] - 302s 2s/step - loss: 0.2330 - accuracy: 0.8949
Epoch 8/10
160/160 [==============================] - 314s 2s/step - loss: 0.1968 - accuracy: 0.9162
Epoch 9/10
160/160 [==============================] - 301s 2s/step - loss: 0.2628 - accuracy: 0.8850
Epoch 10/10
160/160 [==============================] - 303s 2s/step - loss: 0.2086 - accura

In [35]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

In [36]:
# Метрики качества на тестовой выборке
print('LOSS', loss)
print('ACCURACY', accuracy)

LOSS 0.2906246483325958
ACCURACY 0.8849999904632568


### Точночть 88.5%

# Вывод

В результате выполнения данной работы был реализован классификатор, показывающий точность 71%, также был использован VGG16 классификатор, показывающий лучшую в сравнении с собственной реализацией точность - 88.5%.